In [ ]:
from __future__ import annotations

import re
import sqlite3

import pandas as pd

conn = sqlite3.connect("consulates.sqlite")

In [ ]:
all_months = pd.read_pickle("all_months.pkl")
DATA_START_DATE = all_months["Month"].min()
DATA_END_DATE = all_months["Month"].max()
COVID_START_DATE = pd.Timestamp("20200301")
NORMAL_MONTHS = pd.date_range(
    start=DATA_START_DATE, end=COVID_START_DATE, freq="MS", name="Month", closed="left"
)
COVID_MONTHS = pd.date_range(
    start=COVID_START_DATE, end=DATA_END_DATE, freq="MS", name="Month"
)
NORMAL_MONTHS, COVID_MONTHS

In [ ]:
# fmt:off
all_months["Visa Class"] = all_months["Visa Class"].replace(
    to_replace={
        "B1": "B1/B2",
        "B2": "B1/B2",

        "BBBCC": "BBBCC/BBBCV",
        "BBBCV": "BBBCC/BBBCV",

        # conditionals based on https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/Immigrant%20Visa%20Symbols.pdf
        "CR1": "CR1/IR1",
        "CR2": "CR2/IR2",
        "IR1": "CR1/IR1",
        "IR2": "CR2/IR2",

        "CX1": "CX1/FX1",
        "CX2": "CX2/FX2",
        "CX3": "CX3/FX3",
        "FX1": "CX1/FX1",
        "FX2": "CX2/FX2",
        "FX3": "CX3/FX3",

        "C21": "C21/F21",
        "C22": "C22/F22",
        "C23": "C23/F23",
        "C24": "C24/F24",
        "C25": "C25/F25",
        "F21": "C21/F21",
        "F22": "C22/F22",
        "F23": "C23/F23",
        "F24": "C24/F24",
        "F25": "C25/F25",

        "C31": "C31/F31",
        "C32": "C32/F32",
        "C33": "C33/F33",
        "F31": "C31/F31",
        "F32": "C32/F32",
        "F33": "C33/F33",

        "CW1": "CW1/IW1",
        "CW2": "CW2/IW2",
        "IW1": "CW1/IW1",
        "IW2": "CW2/IW2",

        "E21": "E2",
        "E22": "E2",
        "E23": "E2",
        "E2C": "E2",

        "E31": "E3",
        "E32": "E3",
        "E33": "E3",
        "E34": "E3",
        "E35": "E3",
        "E3D": "E3",
        "E3P": "E3",

        "AM": "AM1",
        "BX": "BX1",
        "C2A": "C2",
        "CX": "C21/F21",
        "DV": "DV1",
        "E4": "SE1",
        "EW": "EW3",
        "F2A": "C21/F21",
        "F2B": "C24/F24",
        "F3": "C31/F31",
        "F4": "F41",
        "FX": "C21/F21",
        "I5": "I51",
        "IW": "CW1/IW1",
        "SD": "SD1",
        "SE": "SE1",
        "SI": "SI1",
        "SM": "SM1",
        "SQ": "SQ1",
        "SR": "SR1",
        "SU": "U1",
    }
)
# fmt:on

all_months["Post"] = all_months["Post"].replace(
    to_replace={
        "AIT Taipei": "Taipei",
        "Dar Es Salaam": "Dar es Salaam",
        "Mumbai (Bombay)": "Mumbai",
        "Guatemala": "Guatemala City",
        "Panama": "Panama City",
        "Kuwait": "Kuwait City",
        "Port Au Prince": "Port-au-Prince",
        "Port Of Spain": "Port of Spain",
        "Rio De Janeiro": "Rio de Janeiro",
        "Tijuana Tpf": "Tijuana",
        "Chennai ( Madras)": "Chennai",
        "Chennai (Madras)": "Chennai",
    }
)
all_months = all_months.groupby(
    ["Post", "Visa Class", "Month"], as_index=False, sort=False
)["Issuances"].sum()
all_months

In [ ]:
def slugify(text: str) -> str:
    text = text.strip().lower()
    text = re.sub(r"[\s_]", "-", text)
    text = re.sub(r"[^\w-]", "", text)
    return text


all_months["Post Slug"] = all_months["Post"].apply(slugify)
all_months["Visa Class Slug"] = all_months["Visa Class"].apply(slugify)

post_slugs = pd.DataFrame(
    all_months[["Post Slug", "Post"]].drop_duplicates().set_index("Post Slug")
)
post_slugs.to_sql("post_slugs", conn, if_exists="replace")

visa_slugs = pd.DataFrame(
    all_months[["Visa Class Slug", "Visa Class"]]
    .drop_duplicates()
    .set_index("Visa Class Slug")
)

visa_descriptions_sheet_url = "https://docs.google.com/spreadsheets/d/1GLtqb3PVXFriG7lanunfYJrpUE7_e0_L-PEN2spBQbY/export?format=csv"

visa_descriptions = pd.read_csv(visa_descriptions_sheet_url).set_index(
    "Visa Class Slug"
)

visa_slugs = visa_slugs.join(visa_descriptions, on="Visa Class Slug")
visa_slugs.to_sql("visa_slugs", conn, if_exists="replace")

In [ ]:
def get_baseline(all_months: pd.DataFrame) -> pd.DataFrame:
    all_months = all_months.set_index("Month")
    return all_months[["Issuances"]].reindex(index=NORMAL_MONTHS, fill_value=0).mean()


baselines = all_months.groupby(["Post Slug", "Visa Class Slug"], sort=False).apply(
    get_baseline
)
baselines

In [ ]:
all_months.loc[all_months["Post Slug"] == "guayaquil"].loc[
    all_months["Visa Class Slug"] == "f3"
]

In [ ]:
baselines.loc[("budapest", "cr1ir1")]

In [ ]:
baselines.to_pickle("baselines.pkl")
baselines.to_sql("baselines", conn, if_exists="replace")

In [ ]:
def get_backlog(all_months: pd.DataFrame) -> pd.DataFrame:
    all_months = all_months.reset_index()
    key = tuple(all_months.loc[0, ["Post Slug", "Visa Class Slug"]])
    baseline = baselines.loc[key][0]

    all_months = all_months.set_index("Month")

    normal_months = all_months[["Issuances"]].reindex(index=NORMAL_MONTHS, fill_value=0)

    covid_months = all_months[["Issuances"]].reindex(index=COVID_MONTHS, fill_value=0)
    covid_months["Actual Progress"] = covid_months["Issuances"].cumsum()
    covid_months["Expected Delta"] = covid_months.index.month.map(
        lambda month: normal_months.loc[normal_months.index.month == month][
            "Issuances"
        ].mean()
    )
    covid_months["Expected Progress"] = covid_months["Expected Delta"].cumsum()
    covid_months["Backlog"] = (
        covid_months["Actual Progress"] - covid_months["Expected Progress"]
    )
    covid_months["Months Ahead"] = covid_months["Backlog"] / baseline
    covid_months = covid_months.drop(columns=["Actual Progress", "Expected Progress"])

    return pd.concat([normal_months, covid_months])[
        ["Issuances", "Backlog", "Months Ahead", "Expected Delta"]
    ]


backlogs = all_months.groupby(["Post Slug", "Visa Class Slug"], sort=False).apply(
    get_backlog
)

In [ ]:
backlogs.to_pickle("backlogs.pkl")

In [ ]:
backlogs.to_sql("backlogs", conn, if_exists="replace")